In [2]:
pip install free-fermion-lib

Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip uninstall qiskit qiskit-nature qiskit-algorithms -y
!pip install "qiskit[finance,visualization]" qiskit-nature


Found existing installation: qiskit 2.0.1
Uninstalling qiskit-2.0.1:
  Successfully uninstalled qiskit-2.0.1
Found existing installation: qiskit-nature 0.7.2
Uninstalling qiskit-nature-0.7.2:
  Successfully uninstalled qiskit-nature-0.7.2
Found existing installation: qiskit-algorithms 0.3.1
Uninstalling qiskit-algorithms-0.3.1:
  Successfully uninstalled qiskit-algorithms-0.3.1
  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached qiskit_algorithms-0.3.1-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 20.8 MB/s eta 0:00:00a 0:00:01
Using cached qiskit_nature-0.7.2-py3-none-any.whl (2.2 MB)
Using cached qiskit_algorithms-0.3.1-py3-none-any.whl (310 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 32.6 MB/s eta 0:00:00 0:00:01
  Created wheel for pylatexenc: filename=pyla

In [1]:
import numpy as np
import ff





In [2]:
# Generate a Gaussian state
n_sites = 6
rho = ff.random_FF_state(n_sites)
print(f"Generated state shape: {rho.shape}")

# Verify it's normalized
trace = np.trace(rho)
print(f"Trace (should be 1): {trace}")

Generated state shape: (64, 64)
Trace (should be 1): (1+3.0814879110195774e-32j)


In [3]:
with open("FGS.txt", "w") as f:
    for i, val in enumerate(statevector):
        val = val.item()  # Ensure scalar complex number
        real = val.real
        imag = val.imag
        sign = '+' if imag >= 0 else '-'  # handle sign
        f.write(f"{i+1:4d}  {real:.15f} {sign} {abs(imag):.15f} im\n")


NameError: name 'statevector' is not defined

In [4]:
def purification_rho(rho: np.ndarray) -> np.ndarray:
    # Eigendecomposition of the density matrix
    vals, vecs = np.linalg.eigh(rho)  # Use eigh since rho is Hermitian

    d = rho.shape[0]
    psi = np.zeros(d * d, dtype=np.complex128)

    for k in range(d):
        pk = vals[k]
        if np.isclose(pk, 0):
            continue  # Skip zero-eigenvalue components

        psi_k = vecs[:, k]  # eigenvector |ψ_k⟩
        tr_k = np.zeros(d, dtype=np.complex128)
        tr_k[k] = 1.0       # ancilla basis state |k⟩

        # Tensor product: √pk * (|ψ_k⟩ ⊗ |k⟩)
        psi += np.sqrt(pk) * np.kron(psi_k, tr_k)

    # Normalize the purified state vector
    psi /= np.linalg.norm(psi)

    return psi


## Purified State

In [8]:
statevector = purification_rho(rho)

In [7]:
import numpy as np

# 1. Load complex statevector from a txt file
def load_statevector(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    vec = []
    for line in lines:
        parts = line.strip().split('+')
        real = float(parts[0])
        imag = float(parts[1].replace('im', ''))
        vec.append(real + 1j * imag)
    psi = np.array(vec, dtype=np.complex128)
    psi /= np.linalg.norm(psi)
    return psi


## Checking FGS Fnction

In [13]:
def check_fgs_given_ops(psi, L1, verbose=True):
    N = int(np.log2(len(psi)))  # Number of sites, assuming psi is a valid state vector
    # Break alpha array into annihilation and creation operators
    a_ops = []
    for i in range(N):
        a_ops.append(L1[N+i])
    
    adag_ops = []
    for i in range(N):
        adag_ops.append(L1[i])

    # Build the 2-point correlation matrix       
    C = np.zeros((N, N), dtype=np.complex128)
    for i in range(N):
        for j in range(N):
            C[i, j] = np.vdot(psi, adag_ops[i] @ a_ops[j] @ psi)

    if verbose:
        print("2-point correlation matrix C:")
        print(C)

    # Build the 4-point correlation matrix and also check Wick's theorem
    i, j, k, l = 0, 1, 0, 1
    op = adag_ops[i] @ adag_ops[j] @ a_ops[k] @ a_ops[l]
    four_point = np.vdot(psi, op @ psi)
    wick_value = (C[i, l] * C[j, k] - C[i, k] * C[j, l])
    if verbose:
        print(f"(a†_{i+1} a†_{j+1} a_{k+1} a_{l+1}) = {four_point}")
        print(f"Wick prediction = {wick_value}")
        print(f"Difference = {abs(four_point - wick_value)}")

    return np.abs(four_point - wick_value) < 1e-8


## Sample State

In [30]:
psi_2 = load_statevector('FGS_Gen.txt')
len(psi_2)

4096

## All Zeros State

In [28]:
import numpy as np

def all_zeros_statevector(N):
    state = np.zeros((2**N,), dtype=complex)
    state[0] = 1.0 + 0j  # |0...0⟩ is the first basis state
    return state


psi_3 = all_zeros_statevector(5)
print(psi_3)
len(psi_3)

[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j]


32

##  BELL State

In [87]:
def bell_psi_plus():
    state = np.zeros(4, dtype=complex)
    state[1] = 1 / np.sqrt(2)  # |01⟩
    state[2] = 1 / np.sqrt(2)  # |10⟩
    return state

psi_4 = bell_phi_plus()
print(psi_4)
len(psi_4)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


4

In [69]:
I = np.eye(2)
Z = np.array([[1, 0], [0, -1]])
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def kron(a, b):
    return np.kron(a, b)

a1_dag = kron(I, sigma_plus)
a2_dag = kron(Z, sigma_plus)
a1 = kron(I, sigma_minus)
a2 = kron(Z, sigma_minus)

L4 = [a1_dag, a2_dag, a1, a2]
for i in range(len(L4)):
    print(L4[i])

[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]
[[ 0  1  0  0]
 [ 0  0  0  0]
 [ 0  0  0 -1]
 [ 0  0  0  0]]
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0  0  0  0]
 [ 1  0  0  0]
 [ 0  0  0  0]
 [ 0  0 -1  0]]


In [116]:
check_fgs_given_ops(psi_4, L4)

2-point correlation matrix C:
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
(a†_1 a†_2 a_1 a_2) = 0j
Wick prediction = 0j
Difference = 0.0


np.True_

In [93]:
import numpy as np

I = np.eye(2)
Z = np.array([[1, 0], [0, -1]])
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def kron(a, b, c):
    return np.kron(np.kron(a, b), c)

In [94]:
a0_dag = kron(I, I, sigma_plus)      # site 0 → σ⁺ on rightmost
a1_dag = kron(I, Z, sigma_plus)      # site 1 → σ⁺ middle, Z right
a2_dag = kron(Z, Z, sigma_plus)      # site 2 → σ⁺ left, ZZ right


In [95]:
a0 = kron(I, I, sigma_minus)         # site 0
a1 = kron(I, Z, sigma_minus)         # site 1
a2 = kron(Z, Z, sigma_minus)         # site 2


In [96]:
L6 = [a0_dag, a1_dag, a2_dag, a0, a1, a2]


In [98]:
psi5 = np.zeros(8, dtype=complex)
psi5[1] = 1/np.sqrt(3)  # |001>
psi5[2] = 1/np.sqrt(3)  # |010>
psi5[4] = 1/np.sqrt(3)  # |100>
print(psi5)

[0.        +0.j 0.57735027+0.j 0.57735027+0.j 0.        +0.j
 0.57735027+0.j 0.        +0.j 0.        +0.j 0.        +0.j]


In [99]:
check_fgs_given_ops(psi5, L6)

2-point correlation matrix C:
[[ 0.66666667+0.j  0.        +0.j -0.66666667+0.j]
 [ 0.        +0.j  0.66666667+0.j  0.        +0.j]
 [-0.66666667+0.j  0.        +0.j  0.66666667+0.j]]
(a†_1 a†_2 a_1 a_2) = 0j
Wick prediction = (-0.4444444444444447+0j)
Difference = 0.4444444444444447


np.False_

In [ ]:
import numpy as np
from functools import reduce

def fermionic_dirac_ops(N: int, sparse: bool = False):
    # 1‑qubit building blocks
    I  = np.array([[1, 0],
                   [0, 1]], dtype=complex)
    Z  = np.array([[1, 0],
                   [0,-1]], dtype=complex)
    sigma_plus  = np.array([[0, 1],
                            [0, 0]], dtype=complex)         
    sigma_minus = np.array([[0, 0],
                            [1, 0]], dtype=complex)          

    if sparse:
        from scipy import sparse
        kron = sparse.kron
        I, Z = map(sparse.csr_matrix, (I, Z))
        sigma_plus  = sparse.csr_matrix(sigma_plus)
        sigma_minus = sparse.csr_matrix(sigma_minus)
    else:
        kron = np.kron

    def jw_op(j, single_op):
        
        left  = reduce(kron, [Z]*j, 1) if j > 0 else 1          
        middle = single_op                                      
        right = reduce(kron, [I]*(N-j-1), 1) if j < N-1 else 1   
        return reduce(kron, (left, middle, right))

    creators   = [jw_op(j, sigma_plus)  for j in range(N)]
    annihilators = [jw_op(j, sigma_minus) for j in range(N)]

    return creators + annihilators      



True


In [115]:
L5 = fermionic_dirac_ops(2)

In [24]:
Fer_DI = ff.jordan_wigner_alphas(2)
for i in range(len(Fer_DI)):
    print(Fer_DI[i])

[[0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [1.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 1.-0.j 0.-0.j 0.-0.j]]
[[ 0.-0.j  0.-0.j  0.-0.j  0.-0.j]
 [ 1.-0.j  0.-0.j  0.-0.j  0.-0.j]
 [ 0.-0.j  0.-0.j -0.-0.j -0.-0.j]
 [ 0.-0.j  0.-0.j -1.-0.j -0.-0.j]]
[[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -0.+0.j]]
